# [Hugging Faceを使って事前学習モデルを日本語の感情分析用にファインチューニングしてみた | DevelopersIO](https://dev.classmethod.jp/articles/huggingface-jp-text-classification/)

## GPU を認識できるか確認

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 必要なライブラリのインストール

```
!pip install transformers
!pip install datasets
!pip install fugashi
!pip install ipadic
```

## データセット

Hugging Face のデータセット：[Hugging Face – The AI community building the future.](https://huggingface.co/datasets)

今回は以下のデータセットのうち、日本語のサブセットを使用します。

[tyqiangz/multilingual-sentiments · Datasets at Hugging Face](https://huggingface.co/datasets/tyqiangz/multilingual-sentiments)

In [2]:
from datasets import load_dataset

dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")

Found cached dataset multilingual-sentiments (/root/.cache/huggingface/datasets/tyqiangz___multilingual-sentiments/japanese/1.0.0/b7cdd8874d82679e59432edf79e074f595c4ad26d2e562eba4fb55f361691b07)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
})

取得したデータセットは以下のようにフォーマットを設定することで、データフレームとして扱うことも可能です。

In [4]:
dataset.set_format(type="pandas")
train_df = dataset["train"][:]
train_df.head(5)

,text,source,label
0,普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しか...,amazon_reviews_multi,2
1,十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが...,amazon_reviews_multi,2
2,見た目はかなりおしゃれで気に入りました。2、3回持ち歩いた後いつも通りゼンマイを巻いていたら...,amazon_reviews_multi,2
3,よくある部分での断線はしませんでした ただiphoneとの接続部で接触不良、折れました ip...,amazon_reviews_multi,2
4,プラモデルの塗装剥離に使う為に購入 届いて早速使ってみた 結果 １ヶ月経っても未だに剥離出来...,amazon_reviews_multi,2


どうやらamazonのレビューデータが元になって、そちらに対してラベルが付与されているようです。

`source` と `label` の内訳を見てみましょう。

In [5]:
train_df.value_counts(["source", "label"])

source                label
amazon_reviews_multi  0        40000
                      1        40000
                      2        40000
dtype: int64

各ラベルの意味については、featuresを見れば分かるようになっています。

featuresは、各列の値についての詳細が記載してあります。

In [6]:
dataset["train"].features

{'text': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'label': ClassLabel(names=['positive', 'neutral', 'negative'], id=None)}

このように、labelはClassLabelクラスとなっており、0,1,2がそれぞれ'positive','neutral','negative'に割り当てられていることが分かります。

ClassLabelクラスには、int2strというメソッドがあり、これでラベル名に変換することが可能です。

In [7]:
import numpy as np
from tqdm.auto import tqdm

tqdm.pandas()


def label_int2str(x):
    return dataset["train"].features["label"].int2str(x)


# train_df["label_name"] = train_df["label"].apply(label_int2str)


def _func(x, pbar) -> list[str]:
    result = label_int2str(int(x))
    pbar.update(1)
    return result


input_data = train_df["label"]

with tqdm(total=len(input_data)) as pbar:
    train_df["label_name"] = np.vectorize(_func)(input_data, pbar)

train_df.head()

TypeError: 'type' object is not subscriptable

最後に、データフレームにしていたフォーマットを元に戻しておきます。

In [ ]:
dataset.reset_format()

## モデルの検索

データをトークナイザで処理する前に、使用する事前学習モデルを決める必要があります。理由としては、通常事前学習モデルを作成した時と同じトークナイザを使用する必要があるためと考えられます。

モデルの検索もHugging Faceのページに準備されており、以下から検索が可能です。

<https://huggingface.co/models>

この中で、BERTの日本語版を探し、その中が比較的ダウンロード数の多い以下を使用することにします。

<https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking>

他にも様々な事前学習モデルがありますが、後述するトークナイザの精度などを確認し、問題が無さそうなものを選択しました。

## トークナイザの動作確認

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
sample_text = "\
機械学習のコア部分のロジックを、定型的な実装部分から切り離して\
定義できるようなインターフェースに工夫されています。 \
そのためユーザーは、機械学習のコア部分のロジックの検討に\
集中することができます。\
"

In [ ]:
sample_text_encoded = tokenizer(sample_text)
print(sample_text_encoded)

結果はこのように、`input_ids` と `attention_mask` が含まれます。

input_idsは数字にエンコードされたトークンで、`attention_mask` は後段のモデルで有効なトークンかどうかを判別するためのマスクです。

無効なトークン（例えば、`[PAD]` など）に対しては、`attention_mask` を 0 として処理します。

トークナイザの結果は数字にエンコードされているため、トークン文字列を得るには、`convert_ids_to_tokens` を用います。

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(sample_text_encoded.input_ids)
print(tokens)

結果がこのように得られます。

先頭に##が付加されているものは、サブワード分割されているものです。

また、系列の開始が[CLS]、系列の終了(実際は複数系列の切れ目)が[SEP]という特殊なトークンとなっています。

トークナイザについては以下にも説明があります。

[cl-tohoku/bert-base-japanese-whole-word-masking · Hugging Face](https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking)

> The texts are first tokenized by MeCab morphological parser with the IPA dictionary and then split into subwords by the WordPiece algorithm. The vocabulary size is 32000.

トークン化にIPA辞書を使ったMecabが使用され、サブワード分割にはWordPieceアルゴリズムが使われているようです。

その他、文字列を再構成するには、convert_tokens_to_stringを用います。

In [ ]:
decode_text = tokenizer.convert_tokens_to_string(tokens)
print(decode_text)

## データセット全体のトークン化

データセット全体に処理を適用するには、バッチ単位で処理する関数を定義し、mapを使って実施します。

- `padding=True` でバッチ内の最も長い系列長に合うようpaddingする処理を有効にします。
- `truncation=True` で、後段のモデルが対応する最大コンテキストサイズ以上を切り捨てます。

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

参考までにモデルが対応する最大コンテキストサイズは、以下で確認ができます。

In [ ]:
tokenizer.model_max_length

これをデータセット全体に適用します。

- `batched=True` によりバッチ化され、`batch_size=None` により全体が1バッチとなります。

In [ ]:
%%time
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

In [ ]:
dataset_encoded

データセット全体に適用され、カラムが追加されていることが分かります。

token_types_idは今回使用しませんが、複数の系列がある場合に使用されます。(詳細は下記を参照)

[Glossary](https://huggingface.co/docs/transformers/glossary#token-type-ids)

サンプル単位で結果を確認したい場合は、データフレームなどを使用します。

In [ ]:
import pandas as pd

sample_encoded = dataset_encoded["train"][0]
pd.DataFrame(
    [
        sample_encoded["input_ids"],
        sample_encoded["attention_mask"],
        tokenizer.convert_ids_to_tokens(sample_encoded["input_ids"]),
    ],
    ["input_ids", "attention_mask", "tokens"],
).T

## 分類器の実現方法

テキスト分類のためにはここから、BERTモデルの後段に分類用のヘッドを接続する必要があります。

<img src="https://raw.githubusercontent.com/cm-nakamura-shogo/devio-image/main/huggingface-jp-text-classification/huggingface-jp-text-classification-2.png" alt="drawing" width="200"/>

接続後、テキスト分類を学習する方法に大きく２種類あります。

接続した分類用ヘッドのみを学習
BERTを含むモデル全体を学習(fine-tuning)
前者は高速な学習が可能でGPUなどが利用できない場合に選択肢になり、後者の方がよりタスクに特化できるので高精度となります。

本記事では後者のfine-tuningする方法で実装していきます。

## 分類器の実装

今回のようなテキストを系列単位で分類するタスクには、既にそれ専用のクラスが準備されており、以下で構築が可能です。

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 3

device

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=num_labels
).to(device)

## トレーニングの準備

学習時に性能指標を与える必要があるため、それを関数化して定義しておきます。

In [ ]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

こちらは `EvalPrediction` オブジェクトをうけとる形で実装します。

`EvalPrediciton` オブジェクトは、`predictions` と `label_ids` という属性を持つ `named_tuple` です。

そして学習用のパラメータを `TrainingArguments` クラスを用いて設定します。

In [ ]:
from transformers import TrainingArguments

batch_size = 16
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = "sample-text-classification-bert"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=False,
    log_level="error",
)

## トレーニングの実行

In [ ]:
%%time
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    tokenizer=tokenizer,
)
trainer.train()

## 推論テスト

推論結果は `predict` により得ることができます。

In [ ]:
%%time
preds_output = trainer.predict(dataset_encoded["validation"])

これを混同行列で可視化してみます。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

y_preds = np.argmax(preds_output.predictions, axis=1)
y_valid = np.array(dataset_encoded["validation"]["label"])
labels = dataset_encoded["train"].features["label"].names


def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()


plot_confusion_matrix(y_preds, y_valid, labels)

positive, negativeについては9割以上で正解できていますが、neutralの判別が少し難しくなっていそうです。
またpositiveをnegativeに間違えたり、negativeをpositiveに間違えたりすることは少ないようです。

## モデル保存

保存前にラベル情報を設定しておきます。

In [ ]:
id2label = {}
for i in range(dataset["train"].features["label"].num_classes):
    id2label[i] = dataset["train"].features["label"].int2str(i)

label2id = {}
for i in range(dataset["train"].features["label"].num_classes):
    label2id[dataset["train"].features["label"].int2str(i)] = i

trainer.model.config.id2label = id2label
trainer.model.config.label2id = label2id

`save_model` で保存します。

In [ ]:
model_dir = "work/nlp_tasks/text_classification/Hugging_Faceを使って事前学習モデルを日本語の感情分析用にファインチューニングしてみた/sample-text-classification-bert"
trainer.save_model(model_dir)

保存結果は以下のようなファイル構成となります。

```txt
sample-text-classification-bert
├── config.json
├── pytorch_model.bin
├── special_tokens_map.json
├── tokenizer_config.json
├── training_args.bin
└── vocab.txt
```

モデルやトークナイザの設定ファイル、そしてメインのモデルは pytorch_model.bin として保存されているようです。

## ロードして推論

In [ ]:
%%time
new_tokenizer = AutoTokenizer.from_pretrained(model_dir)

new_model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)

サンプルテキストを推論します。

In [ ]:
%%time
inputs = new_tokenizer(sample_text, return_tensors="pt")

new_model.eval()

with torch.no_grad():
    outputs = new_model(
        inputs["input_ids"].to(device),
        inputs["attention_mask"].to(device),
    )
outputs.logits

logitsを推論ラベルに変換します。

In [ ]:
y_preds = np.argmax(outputs.logits.to("cpu").detach().numpy().copy(), axis=1)


def id2label(x):
    return new_model.config.id2label[x]


y_dash = [id2label(x) for x in y_preds]
y_dash